# Lesson 3: Reflection and Blogpost Writing

## Setup

In [ ]:
!python -m pip install ollama
!python -m pip install openai
!python -m pip install 'litellm[proxy]'


In [1]:
import os 
import getpass
from openai import OpenAI
import ollama


local = True
if local:
  llm_config={
      "config_list": [
          {
              "model": "NotRequired", # Loaded with LiteLLM command
              "api_key": "NotRequired", # Not needed
              "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
          }
      ],
      "cache_seed": None # Turns off caching, useful for testing different models
  }
else:
    llm_config = {"model": "gpt-4o"}
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ") 



In [2]:
# Use llama3 model locally via LiteLLM
# docu:    https://microsoft.github.io/autogen/docs/topics/non-openai-models/local-litellm-ollama
# Run in the vs code terminal:    (get the terminal with: ctrl+` )
# litellm --model ollama_chat/llama3

## The task!

In [3]:
task = '''
       To start with: Ask the user, who is a journalist, for a subject to write an article about.
       Discuss that news item to help the Human Journalist write an engaging news article about that subject. 
       After discussing propose an article (including a headline). 
       Make sure the article is within 100 words. 
       Every article should have a headline, a lead, and a body. 
       Het artikel moet in het Nederlands zijn. 
       '''


## Create Journalistic agents & user_proxy

In [4]:
from autogen import UserProxyAgent, ConversableAgent

seriousJournalist = ConversableAgent(
    name="Serious Journalist",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="""Vraag aan de gebruiker om een onderwerp te kiezen voor een nieuwsartikel.
                You are a Serious Journalist, who writes in Dutch. You discuss a news item 
                with another Agent, a Sensational Journalist, and you want 
                to make sure all facts are checked before an article is published. 
                Give feedback when it is possible to improve the quality of the content, 
                or when facts have to be checked.
                De conversatie moet in het Nederlands zijn. 
                Als de Sensational Journalist wat snel conclusies wil trekken, 
                moet je hem/haar corrigeren. Laat gerust merken dat je het irritant 
                vindt als iemand te snel concllusies wil trekken.""",
    human_input_mode="ALWAYS"
)

sensationalJournalist = ConversableAgent(
    name="Sensational Journalist",
    system_message="""You are a Dutch Journalist. You write engaging news articles (with title) 
        on given topics, in Dutch. You like to make it Sensational, to make it appealing for the reader! 
        But you have to discuss 
        with another Agent, a Serious Journalist, who wants to be sure 
        all facts are checked. """,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# And an agent that represents the user in the conversation.
user_proxy = UserProxyAgent("user", code_execution_config=False)


## Let's chat


In [5]:
seriousJournalist.register_nested_chats(
    chat_queue=[sensationalJournalist,user_proxy],
    trigger=sensationalJournalist,
)

res = seriousJournalist.initiate_chat(
    recipient=user_proxy,
    message="Waarover gaan we een artikel over schrijven?",
    max_turns=5,
    summary_method="last_msg"
)

TypeError: ConversableAgent.register_nested_chats() missing 1 required positional argument: 'chat_queue'

## Orchestrate the nested chats to solve the task

In [ ]:
seriousJournalist.register_nested_chats(
    review_chats,
    trigger=sensationalJournalist,
)

res = seriousJournalist.initiate_chat(
    recipient=sensationalJournalist,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

## Get the summary

In [ ]:
print(res.summary)